In [1]:
%cd /home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/

import numpy as np;
from embeddings.h5_processor import H5DataProcessor, DatasetConfig, ProcessedDataset, AudioSegmentSplitter;
from pathlib import Path;
from embeddings.models import RatingsClassifier;
import yaml;
from sklearn.preprocessing import StandardScaler

/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/.venv/lib64/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen


In [2]:
# Process datasets using H5DataProcessor
processor = H5DataProcessor()
all_train_data = []
all_test_data = []
class_names = set()
with open("universal_music/config.yaml", 'r') as f:
    config = yaml.safe_load(f)


In [3]:
# Process each dataset and split
for dataset_config in config['datasets']:
    print(dataset_config)
    
    dataset = processor.process_h5_file(
        processor.get_embedding_path(DatasetConfig(**dataset_config)),
        DatasetConfig(**dataset_config)
    )
    
    # Split the dataset
    train_data, test_data = processor.get_train_test_split(
        dataset,
    )

    print(dataset)
    
    all_train_data.append(train_data)
    all_test_data.append(test_data)
    class_names.update(dataset.labels)


{'dataset': 'embeddings/NHS', 'subfolder': 'Dance/s15-t15'}
ProcessedDataset(embeddings=array([[ 0.6225814 ,  0.19443417,  0.15496552, ...,  0.4885205 ,
         0.42111373,  0.9090427 ],
       [ 0.55615807,  0.56293535,  0.31450295, ...,  0.10811758,
         0.37352598,  0.6282139 ],
       [ 0.28061938,  0.75397205,  0.119344  , ...,  0.42132023,
         0.26277018,  0.6796056 ],
       ...,
       [ 0.4312954 , -0.08894545, -0.20120752, ...,  0.290231  ,
         0.7523409 ,  0.32112968],
       [ 0.48031878,  0.9571161 ,  0.42678368, ...,  0.05261052,
         0.36954212,  0.8851882 ],
       [ 0.7038944 ,  0.1487031 , -0.45436513, ...,  0.30519223,
         0.91962934,  0.8303722 ]], dtype=float32), labels=['embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings/NHS/Dance', 'embeddings

In [4]:
# Combine datasets
combined_train = ProcessedDataset(
    embeddings=np.vstack([d.embeddings for d in all_train_data]),
    labels=[l for d in all_train_data for l in d.labels],
    filenames=[f for d in all_train_data for f in d.filenames],
    name="combined",
    num_samples=sum(d.num_samples for d in all_train_data)
)

combined_test = ProcessedDataset(
    embeddings=np.vstack([d.embeddings for d in all_test_data]),
    labels=[l for d in all_test_data for l in d.labels],
    filenames=[f for d in all_test_data for f in d.filenames],
    name="combined",
    num_samples=sum(d.num_samples for d in all_test_data)
)


In [17]:
# Create and configure model
model = RatingsClassifier(max_iter=1000)

In [18]:
# Scaling
unique_labels = sorted(set(combined_train.labels + combined_test.labels))
label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

X_train = combined_train.embeddings
y_train = np.array([label_to_idx[label] for label in combined_train.labels])
X_test = combined_test.embeddings
y_test = np.array([label_to_idx[label] for label in combined_test.labels])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# Training
model.fit(X_train_scaled, y_train)

In [20]:
# Predicting ratings on the test set
pred_ratings = model.predict_ratings(X_test_scaled)

The ratings are all either 0 or 3, and not every row even has a rating other than 0.

In [11]:
# Save them to a file to be read by violins.ipynb